In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import pandas_ta as pandas_ta
import numpy as np
import matplotlib.pyplot as plt
from backtesting import Strategy
import plotly.express as px
from datetime import datetime
import MetaTrader5 as mt5

import random
random.seed(42)

from backbone.utils.general_purpose import diff_pips

In [148]:
symbols_path = './backbone/data/backtest/symbols/USDCAD_D.csv'
df = pd.read_csv(symbols_path)
df

,Date,Open,High,Low,Close,Volume
0,2014-01-01 00:00:00,1.06399,1.06480,1.06398,1.06467,4.895600e+08
1,2014-01-02 00:00:00,1.06465,1.06776,1.05882,1.06694,5.565273e+10
2,2014-01-03 00:00:00,1.06694,1.06717,1.06023,1.06308,5.323809e+10
3,2014-01-05 00:00:00,1.06312,1.06399,1.06308,1.06317,7.870000e+08
4,2014-01-06 00:00:00,1.06318,1.06804,1.06087,1.06611,4.378836e+10
...,...,...,...,...,...,...
3122,2023-12-26 00:00:00,1.32521,1.32618,1.31915,1.31963,1.084934e+11
3123,2023-12-27 00:00:00,1.31963,1.32190,1.31767,1.32052,7.631410e+10
3124,2023-12-28 00:00:00,1.32054,1.32349,1.31820,1.32300,1.121052e+11
3125,2023-12-29 00:00:00,1.32296,1.32646,1.31774,1.32461,1.070510e+11


In [149]:
train_start = '2014-01-01'
train_end = '2016-01-01'

test_start = '2016-01-01'
test_end = '2018-01-01'

wfo_start = '2016-01-01'
wfo_end = '2023-01-01'

train_data = df[(df.Date > train_start) & (df.Date < train_end)]
test_data = df[(df.Date > test_start) & (df.Date < test_end)]
wfo_data = df[(df.Date > wfo_start) & (df.Date < wfo_end)]

train_data.loc[:, 'Date'] = pd.to_datetime(train_data.Date)
test_data.loc[:, 'Date'] = pd.to_datetime(test_data.Date)
wfo_data.loc[:, 'Date'] = pd.to_datetime(wfo_data.Date)

df.loc[:, 'Date'] = pd.to_datetime(df.Date)

train_data = train_data.set_index('Date')
test_data = test_data.set_index('Date')
wfo_data = wfo_data.set_index('Date')

df = df.set_index('Date')


# Breakout twist Strategy

## Test Entry: Monkey Exit

In [159]:

import itertools

class BreakoutTwist(Strategy):
    pip_size = 0.0001
    risk=2

    n = 10
    adx_period = 14
    
    q = 50
    bars_percentile = 60
    
    def init(self):
        self.adx = self.I(
            ta.ADX, 
            self.data.High, 
            self.data.Low, 
            self.data.Close, 
            self.adx_period
        )

        self.rsi = self.I(ta.RSI, self.data.Close, 14)
        self.ema_200 = self.I(ta.EMA, self.data.Close, timeperiod=200)
        self.ema_50 = self.I(ta.EMA, self.data.Close, timeperiod=50)
        self.ema_26 = self.I(ta.EMA, self.data.Close, timeperiod=26)
        self.engulfing = self.I(ta.CDLENGULFING, self.data.Open, self.data.High, self.data.Low, self.data.Close)


        self.random_time = None 

    def next(self):
        actual_close = self.data.Close[-1]

        if self.position:
            first_trade = self.trades[0]
            today = self.data.index[-1].tz_localize('UTC').tz_convert('UTC')
            time_in_position = (today - first_trade.entry_time.tz_localize('UTC').tz_convert('UTC'))
            time_in_position = time_in_position.days
            
            percentile = np.percentile(self.data.Close[-self.bars_percentile:-1], q=self.q)
            
            if self.position.is_long and (actual_close < percentile or self.engulfing == -100):
                self.position.close()
                
            elif self.position.is_short and (actual_close > percentile or self.engulfing == 100):
                self.position.close()

            # elif time_in_position >= self.random_time:
            #     self.position.close()
            #     self.random_time = None
  
        else: 
            max_high = max(self.data.High[-self.n:-2])
            min_low = min(self.data.Low[-self.n:-2])


            if self.adx < 20 and actual_close >= max_high:
                self.buy(size=self.risk/100)
                self.random_time = 20


            elif self.adx < 20 and actual_close <= min_low:
                self.sell(size=self.risk / 100)
                self.random_time = 20



metrics = pd.DataFrame()

n_bars = [3, 4, 5, 8, 10]
qs = [25, 50, 75]
bars_percentile = [30, 60, 90]

parameter_combinations = list(itertools.product(
    n_bars, qs, bars_percentile
))

for n, q, bar_percentile in parameter_combinations:
    bt_train = Backtest(
        train_data, 
        BreakoutTwist,
        commission=0.0002, 
        cash=15_000, 
        margin=1/30
    )

    stats = bt_train.run(
        n=n,
        q=q,
        bars_percentile=bar_percentile
    )

    # bt_train.plot(filename='./RsiBBands.html', resample=False)

    win_rate = stats['Win Rate [%]']
    avg_trade_perc = stats['Avg. Trade [%]']

    actual_metrics = pd.DataFrame(
        {
            'winning_rate':[win_rate], 
            'avg_trade_perc':[avg_trade_perc], 
            'n_candles':[n], 
            'q':[q], 
            'bars_percentile':[bar_percentile], 
        }
    )

    metrics = pd.concat([metrics, actual_metrics])

print(stats)
metrics.sort_values(by='winning_rate', ascending=False)

# bt_train = Backtest(
#     train_data, 
#     BreakoutTwist,
#     commission=0.0002, 
#     cash=15_000, 
#     margin=1/30
# )

# stats = bt_train.run(
#     n=30,
#     adx_period=14
# )
# stats

Start                     2014-01-01 00:00:00
End                       2015-12-31 00:00:00
Duration                    729 days 00:00:00
Exposure Time [%]                   26.837061
Equity Final [$]                 15439.207004
Equity Peak [$]                  15654.571121
Return [%]                           2.928047
Buy & Hold Return [%]               29.923826
Return (Ann.) [%]                     1.16855
Volatility (Ann.) [%]                2.606086
Sharpe Ratio                         0.448393
Sortino Ratio                        0.675834
Calmar Ratio                         0.347483
Max. Drawdown [%]                   -3.362897
Avg. Drawdown [%]                   -1.109431
Max. Drawdown Duration      335 days 00:00:00
Avg. Drawdown Duration       86 days 00:00:00
# Trades                                   13
Win Rate [%]                        46.153846
Best Trade [%]                       7.306661
Worst Trade [%]                      -1.32972
Avg. Trade [%]                    

,winning_rate,avg_trade_perc,n_candles,q,bars_percentile
0,55.555556,1.459720,8,25,60
0,55.555556,1.334332,10,50,90
0,55.555556,1.459720,10,25,60
0,55.555556,1.459720,5,25,60
0,55.555556,1.334332,3,50,90
0,55.555556,1.459720,3,25,60
0,55.555556,1.334332,8,50,90
0,55.555556,1.334332,4,50,90
0,55.555556,1.334332,5,50,90
0,55.555556,1.459720,4,25,60


In [164]:
bt_train = Backtest(
    test_data, 
    BreakoutTwist,
    commission=0.0002, 
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run(
    n=3,
    q=25,
    bars_percentile=90
)
stats

Start                     2016-01-03 00:00:00
End                       2017-12-29 00:00:00
Duration                    726 days 00:00:00
Exposure Time [%]                   22.596154
Equity Final [$]                 15015.333302
Equity Peak [$]                  15265.389386
Return [%]                           0.102222
Buy & Hold Return [%]               -9.415521
Return (Ann.) [%]                    0.041269
Volatility (Ann.) [%]                1.898298
Sharpe Ratio                          0.02174
Sortino Ratio                        0.030837
Calmar Ratio                         0.013792
Max. Drawdown [%]                   -2.992191
Avg. Drawdown [%]                   -0.848629
Max. Drawdown Duration      242 days 00:00:00
Avg. Drawdown Duration       51 days 00:00:00
# Trades                                   12
Win Rate [%]                        33.333333
Best Trade [%]                       5.056788
Worst Trade [%]                     -2.102245
Avg. Trade [%]                    

In [165]:
bt_train.plot(filename='./RsiBBands.html', resample=False)


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

GridPlot(id='p126086', ...)

## Test Exit: Monkey Entry

In [ ]:

trade_frecuency = stats['# Trades'] / train_data.shape[0]
long_frecuency = stats._trades[stats._trades['Size'] > 0].shape[0] / stats['# Trades']
short_frecuency = stats._trades[stats._trades['Size'] < 0].shape[0] / stats['# Trades']

def random_boolean(prob_true=0.5):
    return random.choices([True, False], weights=[prob_true, 1 - prob_true], k=1)[0]


class VwapExit(Strategy):
    pip_size = 0.0001
    sl_pips = 15
    rr = 1.5
    risk = 1

    adx_period = 5
    rsi_period = 3

    def init(self):
        self.rsi = self.I(ta.RSI, self.data.Close, 14)

        self.random = None

    def next(self):
        close_prices = self.data.Close
        actual_close = close_prices[-1]
       
        if self.position:
            if self.position.is_long and self.rsi > 70:
                self.position.close()
                
            elif self.position.is_short and self.rsi < 30:
                self.position.close()

        else: 
            trade = random_boolean(prob_true=trade_frecuency)

            if trade:
                long = random_boolean(prob_true=long_frecuency)

                if long:
                    sl = actual_close - self.sl_pips * self.pip_size
                    account_risk = self.equity * (self.risk / 100)
                    units = round(account_risk / (self.pip_size * self.sl_pips))
                    self.buy(sl=sl, size=units)

                else:
                    sl = actual_close + self.sl_pips * self.pip_size
                    account_risk = self.equity * (self.risk / 100)
                    
                    # Calculate lot size in units
                    units = round(account_risk / (self.pip_size * self.sl_pips))
                    self.sell(sl=sl, size=units)


metrics = pd.DataFrame()

for x in range(0, 10):
    bt_train = Backtest(
        train_data, 
        VwapExit, 
        cash=15_000, 
        margin=1/30
    )

    stats = bt_train.run(
    )

    # bt_train.plot(filename='./RsiBBands.html', resample=False)

    equity = stats['Equity Final [$]']
    return_ = stats['Return [%]']
    sharpe_ratio = stats['Sharpe Ratio']

    actual_metrics = pd.DataFrame(
        {
            'equity':[equity], 
            'return_':[return_], 
            'sharpe_ratio':[sharpe_ratio], 
        }
    )

    metrics = pd.concat([metrics, actual_metrics])



In [ ]:
metrics.return_.std()

## Full system

In [ ]:
import itertools

class VwapRsiFull(Strategy):
    pip_size = 0.0001
    sl_pips = 10
    rr = 1
    risk = 1

    n_candles = 7 
    distance = 1

    rsi_up_threshold=70
    rsi_down_threshold=30


    def init(self):
        self.vwap = self.I(
            pandas_ta.vwap, 
            pd.Series(self.data.High, index=self.data.index),
            pd.Series(self.data.Low, index=self.data.index),
            pd.Series(self.data.Close, index=self.data.index),
            pd.Series(self.data.Volume, index=self.data.index),
        ) 

        self.ema_50 = self.I(ta.EMA, self.data.Close, timeperiod=200) 
        
        self.rsi = self.I(ta.RSI, self.data.Close, 14)


    def next(self):
        actual_close = self.data.Close[-1]
       
        if self.position:
            pass

        else: 

            actual_vwap = self.vwap[-1]
            distance_vwap = diff_pips(actual_vwap, actual_close, pip_value=self.pip_size, absolute=True)

            n_candles_under_vwap = True
            n_candles_up_vwap = True

            for x in range(1, self.n_candles):
                if self.data.Close[-x] > self.vwap[-x]:
                    n_candles_under_vwap = False

                if self.data.Close[-x] < self.vwap[-x]:
                    n_candles_up_vwap = False

            if distance_vwap <= self.distance and n_candles_up_vwap: 
                sl = actual_close - self.sl_pips * self.pip_size
                tp = actual_close + self.rr * self.sl_pips * self.pip_size
                
                account_risk = self.equity * (self.risk / 100)
                units = round(account_risk / (self.pip_size * self.sl_pips))
                self.buy(sl=sl, size=units, tp=tp)

            elif distance_vwap <= self.distance and n_candles_under_vwap: 
                sl = actual_close + self.sl_pips * self.pip_size
                tp = actual_close - self.rr * self.sl_pips * self.pip_size

                account_risk = self.equity * (self.risk / 100)
                
                # Calculate lot size in units
                units = round(account_risk / (self.pip_size * self.sl_pips))
                self.sell(sl=sl, size=units, tp=tp)


bt_train = Backtest(
    train_data, 
    VwapRsiFull, 
    commission=0.0002,
    cash=15_000, 
    margin=1/30
)


stats = bt_train.run(rr=2, sl_pips=15)

# stats = bt_train.optimize(
#     distance=[1, 2, 3],
#     n_candles=[6, 8,  10, 12],
#     sl_pips=[5, 8, 12, 15],
#     rsi_up_threshold=[60, 70, 80],
#     rsi_down_threshold=[20, 30, 40],
# )

bt_train.plot(filename='./RsiBBands.html', resample=False)

stats


In [ ]:
stats._strategy.__dict__

# WFO

In [23]:
import pickle
from wfo_utils.utils import walk_forward, plot_full_equity_curve, plot_stats

In [152]:

lookback_bars = 365
validation_bars = 180
warmup_bars = 200 

params = {
    'n': [3, 4, 5],
    'q': [30, 60, 90],
    'adx_period': [14],
    'maximize': 'Return [%]' 
}


stats = walk_forward(
    BreakoutTwist,
    wfo_data, 
    lookback_bars=lookback_bars,
    validation_bars=validation_bars,
    warmup_bars=warmup_bars, 
    params=params,
    commission=0.0002, 
    margin=1/30, 
    cash=15_000
)

train from 2016-01-03 00:00:00 to 2017-03-02 00:00:00


  0%|          | 0/9 [00:00<?, ?it/s]

validate from 2016-07-13 00:00:00 to 2017-09-28 00:00:00
equity final: 15437.483803894002
train from 2016-07-31 00:00:00 to 2017-09-28 00:00:00


  0%|          | 0/9 [00:00<?, ?it/s]

validate from 2017-02-08 00:00:00 to 2018-04-27 00:00:00
equity final: 15589.635613160002
train from 2017-02-26 00:00:00 to 2018-04-27 00:00:00


...

validate from 2020-07-23 00:00:00 to 2021-10-10 00:00:00
equity final: 15192.469095858018
train from 2020-08-10 00:00:00 to 2021-10-10 00:00:00


  0%|          | 0/9 [00:00<?, ?it/s]

validate from 2021-02-19 00:00:00 to 2022-05-08 00:00:00
equity final: 14855.267172974023
train from 2021-03-09 00:00:00 to 2022-05-08 00:00:00


  0%|          | 0/9 [00:00<?, ?it/s]

validate from 2021-09-17 00:00:00 to 2022-12-04 00:00:00
equity final: 14954.143335614024


In [155]:
for x in range(0, len(stats)):
    plot_stats(df, stats[x], BreakoutTwist, plot=True)

Start                     2016-07-13 00:00:00
End                       2017-09-28 00:00:00
Duration                    442 days 00:00:00
Exposure Time [%]                   28.684211
Equity Final [$]                 15437.483804
Equity Peak [$]                  15718.208644
Return [%]                           2.916559
Buy & Hold Return [%]               -4.148748
Return (Ann.) [%]                     1.92476
Volatility (Ann.) [%]                2.109748
Sharpe Ratio                         0.912317
Sortino Ratio                        1.504997
Calmar Ratio                         1.077415
Max. Drawdown [%]                   -1.786461
Avg. Drawdown [%]                   -0.440566
Max. Drawdown Duration       55 days 00:00:00
Avg. Drawdown Duration       12 days 00:00:00
# Trades                                    8
Win Rate [%]                             37.5
Best Trade [%]                        5.78074
Worst Trade [%]                      -1.25249
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

Start                     2017-02-08 00:00:00
End                       2018-04-27 00:00:00
Duration                    443 days 00:00:00
Exposure Time [%]                    7.631579
Equity Final [$]                 15589.635613
Equity Peak [$]                  15648.836413
Return [%]                             0.9856
Buy & Hold Return [%]               -2.440212
Return (Ann.) [%]                    0.652528
Volatility (Ann.) [%]                 0.84403
Sharpe Ratio                          0.77311
Sortino Ratio                        1.233042
Calmar Ratio                         1.014038
Max. Drawdown [%]                   -0.643494
Avg. Drawdown [%]                   -0.228109
Max. Drawdown Duration      110 days 00:00:00
Avg. Drawdown Duration       26 days 00:00:00
# Trades                                    2
Win Rate [%]                             50.0
Best Trade [%]                       2.043446
Worst Trade [%]                      -0.39584
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

Start                     2017-09-06 00:00:00
End                       2018-11-23 00:00:00
Duration                    443 days 00:00:00
Exposure Time [%]                        30.0
Equity Final [$]                 15220.353894
Equity Peak [$]                   15732.46564
Return [%]                          -2.368764
Buy & Hold Return [%]                8.132274
Return (Ann.) [%]                   -1.577199
Volatility (Ann.) [%]                1.721074
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -3.88621
Avg. Drawdown [%]                   -1.400445
Max. Drawdown Duration      149 days 00:00:00
Avg. Drawdown Duration       57 days 00:00:00
# Trades                                   11
Win Rate [%]                        18.181818
Best Trade [%]                       0.726078
Worst Trade [%]                       -1.6543
Avg. Trade [%]                    

...

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

Start                     2021-02-19 00:00:00
End                       2022-05-08 00:00:00
Duration                    443 days 00:00:00
Exposure Time [%]                   31.578947
Equity Final [$]                 14855.267173
Equity Peak [$]                  15345.263604
Return [%]                          -2.219533
Buy & Hold Return [%]                2.438386
Return (Ann.) [%]                   -1.477458
Volatility (Ann.) [%]                1.945505
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.945475
Avg. Drawdown [%]                   -1.021262
Max. Drawdown Duration      110 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   17
Win Rate [%]                        23.529412
Best Trade [%]                       0.614967
Worst Trade [%]                     -1.020956
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

Start                     2021-09-17 00:00:00
End                       2022-12-04 00:00:00
Duration                    443 days 00:00:00
Exposure Time [%]                   27.368421
Equity Final [$]                 14954.143336
Equity Peak [$]                  15294.811344
Return [%]                           0.665597
Buy & Hold Return [%]                5.330774
Return (Ann.) [%]                    0.440902
Volatility (Ann.) [%]                2.718079
Sharpe Ratio                         0.162211
Sortino Ratio                        0.222318
Calmar Ratio                         0.196822
Max. Drawdown [%]                   -2.240106
Avg. Drawdown [%]                   -0.664168
Max. Drawdown Duration      113 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                   12
Win Rate [%]                        41.666667
Best Trade [%]                       7.132274
Worst Trade [%]                     -2.822145
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single forma

In [154]:
plot_full_equity_curve(
    df, 
    stats, 
    warmup_bars=warmup_bars,
    lookback_bars=lookback_bars, 
    overlay_price=True)

In [ ]:
trades = test_stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count'], 'Duration':'max'})
trades.columns = trades.columns.droplevel(0)
trades = trades.reset_index().rename(columns={'count':'ammount_trades'})
trades = trades.rename(columns={'sum':'profit'})
trades = trades.rename(columns={'max':'minutes_in_trade'})
trades

In [ ]:
trades.minutes_in_trade.describe()
